<a href="https://colab.research.google.com/github/scaairesearch/da_cv_fer/blob/main/evaluating_synthetic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title loading libraries
!git clone https://github.com/scaairesearch/da_cv_fer.git
import os
os.getcwd()
os.chdir('da_cv_fer')

Cloning into 'da_cv_fer'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 352 (delta 70), reused 67 (delta 32), pack-reused 209
Receiving objects: 100% (352/352), 22.11 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [7]:
!pip install -q -r requirements.txt # -q is for quiet install
# for some reason the below does not get installed with requirements
!pip install -q opendatasets
!pip install -q facenet_pytorch

ERROR: Could not find a version that satisfies the requirement pywin32==306 (from versions: none)
ERROR: No matching distribution found for pywin32==306


In [8]:
#@title loading libraries
from da_cv_fer.data_config import DataConfig
from da_cv_fer.ds_sfew import DatasetSFEW
from da_cv_fer.ds_sfew_crop import DatasetSFEWCROP
from da_cv_fer.ds_expw_crop import EXPWCROP
from da_cv_fer.ds_expw import EXPW
from da_cv_fer.utils import *
from da_cv_fer.model_DANN import DANN
from da_cv_fer.model_DANN_Bespoke import DANNBespoke
from da_cv_fer.run_config import RunConfig
from da_cv_fer.train import *
import os
import zipfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm # for beautiful model training updates

import matplotlib.pyplot as plt # for plots

import time
from pathlib import Path

from copy import deepcopy
import numpy as np


In [9]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [17]:

# Mounting Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [20]:
#@title Loading models to evaluate
non_dann_model_sfew_name =  'non_dann_sfew_expw_03_03.pt'
dataconfig_inference = DataConfig()

In [22]:
# NON DANN

non_dann_model_inference  = DANNBespoke(num_classes=7,backbone='mobilenet', attention=False, dropout = 0)
# model_dir = Path('/content/gdrive/MyDrive/CV_FER/models/non_dann_sfew_expw')
NON_DANN_SFEW_DIR = Path(dataconfig_inference.MODEL_DIR,"non_dann_sfew_expw") # Path(self.MODEL_DIR,"non_dann_sfew")

try:
    non_dann_model_inference.load_state_dict(torch.load(Path(NON_DANN_SFEW_DIR, non_dann_model_sfew_name),
                                                        map_location=torch.device(device)))
    print(f'{non_dann_model_sfew_name} locked and loaded')
except Exception as e:
    print(f' problem in loading {non_dann_model_sfew_name} {e}')

feature extractor backbone created using mobilenet model
non_dann_sfew_expw_03_03.pt locked and loaded


In [23]:
#@title Get SFEW Dataset

sfew = DatasetSFEWCROP()
sfew_train_loader, sfew_val_loader = sfew.get_dataloader()
print(f' length of SFEW dataset : {len(sfew_train_loader.dataset)}')

 self.BASE_PATH -dataset, 
 self.EXTRACT_DIR-dataset/sfew,
 self.ZIP_FILE_PATH - dataset/SFEW_2.zip 
Directory dataset/sfew created successfully.
No files (including zip file) found in dataset/sfew.Copying file
Starting File copying from /content/gdrive/MyDrive/CV_FER/dataset/SFEW_2.zip to dataset/sfew
File copied successfully from /content/gdrive/MyDrive/CV_FER/dataset/SFEW_2.zip to dataset/sfew
Initiating extraction of dataset/sfew/SFEW_2.zip to dataset/sfew
...completed for dataset/sfew/SFEW_2.zip
dataset/sfew/Train Sad.zip Sad
...completed for dataset/sfew/Train/Sad.zip
dataset/sfew/Train Surprise.zip Surprise
...completed for dataset/sfew/Train/Surprise.zip
dataset/sfew/Train Angry.zip Angry
...completed for dataset/sfew/Train/Angry.zip
dataset/sfew/Train Disgust.zip Disgust
...completed for dataset/sfew/Train/Disgust.zip
dataset/sfew/Train Fear.zip Fear
...completed for dataset/sfew/Train/Fear.zip
dataset/sfew/Train Neutral.zip Neutral
...completed for dataset/sfew/Train/Neutral.

In [36]:
non_dann_model_inference.eval()
all_preds = []
all_gts =[]

with torch.no_grad():
  for batch in sfew_val_loader:
    images, labels = batch[0].to(device), batch[1].to(device)
    class_output,_,_ = non_dann_model_inference(images)
    label_gts = labels.argmax(dim=1)
    label_preds = class_output.argmax(dim=1)
    all_gts.extend(label_gts.cpu().numpy())
    all_preds.extend(label_preds.cpu().numpy())

In [ ]:
labels_map={0:"Angry",
            1:"Disgust",
            2:"Fear",
            3:"Happy",
            4:"Sad",
            5:"Surprise",
            6:"Neutral"}
all_preds = [labels_map[int_label] for int_label in all_preds]
all_gts= [labels_map[int_label] for int_label in all_gts]

all_preds, all_gts

In [45]:
from sklearn.metrics import classification_report
# Calculate performance metrics
accuracy = (np.array(all_preds) == np.array(all_gts)).mean()
classification_rep = classification_report(all_gts, all_preds)

In [46]:
# Print the classification report
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

Accuracy: 0.45584725536992843
Classification Report:
               precision    recall  f1-score   support

       Angry       0.43      0.77      0.55        75
     Disgust       0.23      0.32      0.27        22
        Fear       0.19      0.12      0.14        43
       Happy       0.65      0.71      0.68        72
     Neutral       0.33      0.17      0.23        52
         Sad       0.45      0.48      0.47        84
    Surprise       0.62      0.30      0.40        71

    accuracy                           0.46       419
   macro avg       0.42      0.41      0.39       419
weighted avg       0.46      0.46      0.43       419



In [47]:
all_gts = np.array(all_gts)
all_preds = np.array(all_preds)

# Get unique classes in the dataset
unique_classes = np.unique(all_gts)

# Initialize arrays to store class-wise correct predictions and total samples
class_correct = np.zeros(len(unique_classes))
class_total = np.zeros(len(unique_classes))

# Iterate over each unique class
for i, cls in enumerate(unique_classes):
    # Filter predictions and labels for the current class
    class_preds = all_preds[all_gts == cls]
    class_labels = all_gts[all_gts == cls]

    # Calculate accuracy for the current class
    class_correct[i] = np.sum(class_preds == class_labels)
    class_total[i] = len(class_labels)

# Print accuracy for each class
for i, cls in enumerate(unique_classes):
    if class_total[i] > 0:
        accuracy = 100 * class_correct[i] / class_total[i]
        print(f'Accuracy for class {cls}: {accuracy:.2f}% (Total Correct: {int(class_correct[i])}/{int(class_total[i])})')
    else:
        print(f'Accuracy for class {cls}: N/A (No samples in this class)')

Accuracy for class Angry: 77.33% (Total Correct: 58/75)
Accuracy for class Disgust: 31.82% (Total Correct: 7/22)
Accuracy for class Fear: 11.63% (Total Correct: 5/43)
Accuracy for class Happy: 70.83% (Total Correct: 51/72)
Accuracy for class Neutral: 17.31% (Total Correct: 9/52)
Accuracy for class Sad: 47.62% (Total Correct: 40/84)
Accuracy for class Surprise: 29.58% (Total Correct: 21/71)
